# Desafio Semantix - Logs Nasa #

In [1]:
from pyspark.sql import SQLContext, Column
from pyspark import SparkContext
from pyspark.sql.functions import *

In [2]:
sc = SparkContext(appName="RequestsNasa")

In [3]:
sqlContext = SQLContext(sc)

## Arquivos de Log

In [4]:
arq_log_Aug95 = sqlContext.read.text("access_log_Aug95")
arq_log_Jul95 = sqlContext.read.text("access_log_Jul95")

In [6]:
arq_log_Aug95.show(5, truncate=False)
arq_log_Jul95.show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                     |
+--------------------------------------------------------------------------------------------------------------------------+
|in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839|
|uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0                                                   |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0                          |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0                        |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0                           |


In [7]:
df_split_Aug95 = arq_log_Aug95.select(regexp_extract('value', r'^([^\s]+\s)', 1).alias('host'),
                        regexp_extract('value', r'^.*\[(\d\d/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]', 1).alias('data'),
                        regexp_extract('value', r'^.*"\w+\s+([^\s]+)\s+HTTP.*"', 1).alias('path'),
                        regexp_extract('value', r'^.*"\s+([^\s]+)', 1).cast('integer').alias('status'),
                        regexp_extract('value', r'^.*\s+(\d+)$', 1).cast('integer').alias('tamanho'))

df_split_Jul95 = arq_log_Jul95.select(regexp_extract('value', r'^([^\s]+\s)', 1).alias('host'),
                        regexp_extract('value', r'^.*\[(\d\d/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]', 1).alias('data'),
                        regexp_extract('value', r'^.*"\w+\s+([^\s]+)\s+HTTP.*"', 1).alias('path'),
                        regexp_extract('value', r'^.*"\s+([^\s]+)', 1).cast('integer').alias('status'),
                        regexp_extract('value', r'^.*\s+(\d+)$', 1).cast('integer').alias('tamanho'))

In [8]:
df_split_Aug95.show(2, truncate=False)
df_split_Jul95.show(2, truncate=False)

+------------------+--------------------------+-----------------------------------------------+------+-------+
|host              |data                      |path                                           |status|tamanho|
+------------------+--------------------------+-----------------------------------------------+------+-------+
|in24.inetnebr.com |01/Aug/1995:00:00:01 -0400|/shuttle/missions/sts-68/news/sts-68-mcc-05.txt|200   |1839   |
|uplherc.upl.com   |01/Aug/1995:00:00:07 -0400|/                                              |304   |0      |
+------------------+--------------------------+-----------------------------------------------+------+-------+
only showing top 2 rows

+---------------------+--------------------------+-------------------+------+-------+
|host                 |data                      |path               |status|tamanho|
+---------------------+--------------------------+-------------------+------+-------+
|199.72.81.55         |01/Jul/1995:00:00:01 -0400|/

## df1 = Log Aug_95 & df2 = Log Jul_95

In [9]:
df1 = df_split_Aug95.alias('df1')
df2 = df_split_Jul95.alias('df2')

In [11]:
df1_rows = df1.filter(df1['host'].isNull() |
                            df1['data'].isNull() |
                            df1['path'].isNull() |
                            df1['status'].isNull() |
                            df1['tamanho'].isNull())

df2_rows = df2.filter(df2['host'].isNull() |
                            df2['data'].isNull() |
                            df2['path'].isNull() |
                            df2['status'].isNull() |
                            df2['tamanho'].isNull())



In [12]:
def count_null_df1( col_name ):
  return df1.filter( df1[col_name].isNull() ).count()

def count_null_df2( col_name ):
  return df2.filter( df2[col_name].isNull() ).count()

In [14]:
for col_name in df1.columns:
  print( col_name, " : ", count_null_df1( col_name ) )

for col_name in df2.columns:
  print( col_name, " : ", count_null_df2( col_name ) )

host  :  0
data  :  0
path  :  0
status  :  0
tamanho  :  14178
host  :  0
data  :  0
path  :  0
status  :  1
tamanho  :  19727


In [15]:
( df1.filter(
  df1["tamanho"].isNull() )
  .groupby( "status").count() ).show()

( df2.filter(
  df2["tamanho"].isNull() )
  .groupby( "status").count() ).show()

+------+-----+
|status|count|
+------+-----+
|   501|   27|
|   400|   10|
|   403|  171|
|   404|10056|
|   200|   78|
|   302| 3836|
+------+-----+

+------+-----+
|status|count|
+------+-----+
|   501|   14|
|  null|    1|
|   400|    5|
|   403|   54|
|   404|10844|
|   200|   83|
|   302| 8726|
+------+-----+



In [16]:
(arq_log_Aug95.filter(
      ~arq_log_Aug95["value"].rlike('''\d+$'''))
      .show( 5, truncate = False ) )

(arq_log_Jul95.filter(
      ~arq_log_Jul95["value"].rlike('''\d+$'''))
      .show( 5, truncate = False ) )

+---------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                      |
+---------------------------------------------------------------------------------------------------------------------------+
|gw1.att.com - - [01/Aug/1995:00:03:53 -0400] "GET /shuttle/missions/sts-73/news HTTP/1.0" 302 -                            |
|js002.cc.utsunomiya-u.ac.jp - - [01/Aug/1995:00:07:33 -0400] "GET /shuttle/resources/orbiters/discovery.gif HTTP/1.0" 404 -|
|tia1.eskimo.com - - [01/Aug/1995:00:28:41 -0400] "GET /pub/winvn/release.txt HTTP/1.0" 404 -                               |
|itws.info.eng.niigata-u.ac.jp - - [01/Aug/1995:00:38:01 -0400] "GET /ksc.html/facts/about_ksc.html HTTP/1.0" 403 -         |
|grimnet23.idirect.com - - [01/Aug/1995:00:50:12 -0400] "GET /www/software/winvn/winvn.html HTTP/1.0" 404 -           

In [23]:
df1_clean = df1.na.fill( 0 )
df1_clean.cache()

df2_clean = df2.na.fill( 0 )
df2_clean.cache()

DataFrame[host: string, data: string, path: string, status: int, tamanho: int]

# Número de Hosts únicos

In [34]:
df1_clean.select('host').distinct().count()

75060

In [36]:
df2_clean.select('host').distinct().count()

81983

# Total de erros 404

In [38]:
df1_clean.select('status').where(df1_clean['status'] == '404').count()

10056

In [40]:
df2_clean.select('status').where(df2_clean['status'] == '404').count()

10845

# Os 5 URLs que mais causaram erro 404

In [52]:
( df1_clean.filter(
  df1_clean["status"] == '404' )
  .groupby("host").count() ).sort(col("count").desc()).show(5)

+--------------------+-----+
|                host|count|
+--------------------+-----+
|dialip-217.den.mm...|   62|
|piweba3y.prodigy....|   47|
|       155.148.25.4 |   44|
|       maz3.maz.net |   39|
|      gate.barr.com |   38|
+--------------------+-----+
only showing top 5 rows



In [53]:
( df2_clean.filter(
  df2_clean["status"] == '404' )
  .groupby("host").count() ).sort(col("count").desc()).show(5)

+--------------------+-----+
|                host|count|
+--------------------+-----+
|hoohoo.ncsa.uiuc....|  251|
|jbiagioni.npt.nuw...|  131|
|piweba3y.prodigy....|  110|
|piweba1y.prodigy....|   92|
|phaelon.ksc.nasa....|   64|
+--------------------+-----+
only showing top 5 rows



# Quantidades de erros 404 por dia

In [98]:
(df1_clean.filter(
  df1_clean["status"] == '404' )).withColumn('Dia', concat(df1_clean.data.substr(1, 2)
                                   )).distinct().count()

10025

# Total de bytes retornados

In [54]:
total_df1 = df1_clean.groupBy().sum('tamanho')
total_df1.show()

+------------+
|sum(tamanho)|
+------------+
| 26828341424|
+------------+



In [55]:
total_df2 = df2_clean.groupBy().sum('tamanho')
total_df2.show()

+------------+
|sum(tamanho)|
+------------+
| 38695973491|
+------------+

